# Cryptopunks: obtener datos de un cryptopunk

In [191]:
import pandas as pd
import numpy as np


import requests
from bs4 import BeautifulSoup

import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

from selenium.webdriver.common.keys import Keys
from scrapy import Selector
import math

## Acceso a la página de un cryptopunk

In [192]:
# Define the cryptopunt to scrap
# nro = 8348
nro = 5214

url = 'https://www.larvalabs.com/cryptopunks/details/'+str(nro)



In [193]:
# Use BeautifulSoup por webscrapping
html=requests.get(url)
soup = BeautifulSoup(html.content, 'html.parser')
#print(soup)

## Obtener atributos del cryptopunk

In [201]:
cod =soup.find_all('div', class_="col-md-4")
textos = []
for c in cod:
    textos.append(c.find('a').text)
    
atributos = textos[1:]
print('Atributos:',atributos)

Atributos: ['Silver Chain', 'Wild Hair', 'Eye Patch']


## Obtener la lista de transacciones realizadas

In [202]:
text1 = soup.find_all('a', text = '')
#print(text1)

In [203]:
transacciones =[]

In [205]:
transac =soup.find('table', class_="table")
#print(transac)

for tr in transac.find_all('tr'):
    lista = []
    for td in tr.find_all('td'):
        lista.append(td.text.strip())
    transacciones.append(lista)
transacciones = transacciones[1:]
print(transacciones)

[['Bid Withdrawn', '0x6ad931', '', '87Ξ ($207,436)', 'Jan 23, 2022'], ['Bid', '0x6ad931', '', '87Ξ ($207,436)', 'Jan 23, 2022'], ['Bid Withdrawn', 'punksotc…', '', '75Ξ ($277,358)', 'Dec 31, 2021'], ['Offered', '', '', '102.69Ξ ($379,759)', 'Dec 31, 2021'], ['Bid', 'punksotc…', '', '75Ξ ($277,358)', 'Dec 31, 2021'], ['Bid', 'thebinoc…', '', '69Ξ ($255,170)', 'Dec 31, 2021'], ['Bid Withdrawn', 'bundo.et…', '', '21Ξ ($43,351)', 'Jul 23, 2021'], ['Bid', 'bundo.et…', '', '21Ξ ($39,573)', 'Jul 17, 2021'], ['Claimed', '', '0xcc7c33', '', 'Jun 23, 2017'], [], ['Bid', '0x6ad931', '', '95Ξ ($226,510)', 'Jan 23, 2022'], ['Bid Withdrawn', '0x6ad931', '', '87Ξ ($207,436)', 'Jan 23, 2022'], ['Bid', '0x6ad931', '', '87Ξ ($207,436)', 'Jan 23, 2022'], ['Bid Withdrawn', 'punksotc…', '', '75Ξ ($277,358)', 'Dec 31, 2021'], ['Offered', '', '', '102.69Ξ ($379,759)', 'Dec 31, 2021'], ['Bid', 'punksotc…', '', '75Ξ ($277,358)', 'Dec 31, 2021'], ['Bid', 'thebinoc…', '', '69Ξ ($255,170)', 'Dec 31, 2021'], ['Bid

In [206]:
## Obtener el nombre del owner, y saber si está a la venta y su precio

In [207]:
varios =soup.find_all('div', class_="col-md-10 col-md-offset-1")

for linea in varios:
    texto = linea.text
    if texto.strip().startswith('Current Market Status') and texto.find('currently')>0:
        if texto.find("This punk is currently owned by address")>0:
            x = texto.split('address ')[1].split('.',1)[0]
            print('Owner:',x)
            
        if texto.find("This punk has not been listed for sale by its owner")>0:
            sale = False
        elif texto.find('This punk is currently for sale by owner')>0:
            sale = True

        if texto.find("There are currently no bids on this punk.")>0:
            bids = False
        elif texto.find('There is a bid of')>0:
            bids = True
        print('For sale',sale)
        print('Bids:',bids)



Owner: 0xcc7c33
For sale True
Bids: True
